In [1]:
import string
import pandas as pd
import nltk
import re

import time

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/Theo.Henaff/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/Theo.Henaff/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
stopword = nltk.corpus.stopwords.words('english')
wn = nltk.WordNetLemmatizer()
ps = nltk.PorterStemmer()

def clean_text(text):
        
    # remove users tags and url
    text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", text).split())
    # remove punctuation
    text = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    # tokenize
    text = re.split('\W+', text)
    # remove stop-word
    
    text = [word for word in text if word not in stopword]
    # Stemming
    text = [ps.stem(word) for word in text]
    # Lammitization
    text = [wn.lemmatize(word) for word in text]
    # remove empty words
    text = [word for word in text if word != '']
    # rejoin for easier one-hot extraction
    text = ' '.join(text)
    # remove one letters words
    new_text = []
    for w in text.split(" "):
        if len(w) > 1:
            new_text.append(w)
    text = " "
    
    return text.join(new_text)

# Load and preapre the dataset

In [3]:
start_time = time.time()

data = pd.read_csv(
     "training.1600000.processed.noemoticon.csv",
     header=None,
     encoding='latin-1',
     usecols=[0, 5])

# available columns are [0,1,2,4,5]=['polarity', 'id', 'date', 'user', 'text']

data.columns = ['polarity', 'text']
data['polarity'] = pd.to_numeric(data['polarity'], downcast='integer')

# 0 ->[1,0] negative ou 0, 4 ->[0,1] positive ou 1
data.polarity = data.polarity.replace({0: 0, 4: 1})

start_time_2 = time.time()

# Cleanse the data

In [4]:
data['clean_text'] = data['text'].apply(lambda x: clean_text(x)) # ~260s
# data['clean_text'] = data['text'].map(lambda x: clean_text(x)) # ~250s

In [5]:
print("Temps chargement dataset : {}s".format(round(start_time_2 -start_time,2)))
print("Temps nettoyage du dataset : {}s".format(round(time.time() - start_time_2,2)))

Temps chargement dataset : 2.48s
Temps nettoyage du dataset : 254.8s


# Save dataframe to cleaned_data.csv

In [6]:
data.to_csv(r'cleaned_data.csv', index = False)